In [1]:
import os
import utils, segmentation, presentation
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp

In [2]:
ds_train = segmentation.SegmentationDataset('./cut_stenoses_data/train/', 2)
ds_test = segmentation.SegmentationDataset('./cut_stenoses_data/test/', 2)

batch_size = 8
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, drop_last=True)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, shuffle=False, drop_last=False)

In [3]:
len(dl_train), len(dl_test)

(74, 19)

In [4]:
device = torch.device('cuda:1')

In [5]:
model = presentation.create_model().to(device)

In [6]:
optimizer = presentation.create_optimizer(model)

In [7]:
# w = 0.
# for imgs, masks in dl_train:
#     w += masks.sum() / (masks.numel() * len(dl_train))
# loss_fn = nn.CrossEntropyLoss(weight=torch.Tensor([1 / (1-w), 1/ w])).to(device)
loss_fn = nn.CrossEntropyLoss()

In [8]:
for epoch in range(50):
    trn_loss = utils.train(dl_train, loss_fn, model, optimizer)
    val_loss, val_IoUs = segmentation.evaluate(dl_test, loss_fn, model)
    print(epoch, f'{trn_loss:.4f}', f'{val_loss:.4f}', val_IoUs)

0 0.0209 0.0159 tensor([0.])
1 0.0065 0.0206 tensor([0.])
2 0.0057 0.0200 tensor([0.0005])
3 0.0051 0.0177 tensor([0.3104])
4 0.0047 0.0179 tensor([0.3568])
5 0.0043 0.0180 tensor([0.3893])
6 0.0039 0.0200 tensor([0.3762])
7 0.0037 0.0176 tensor([0.4148])
8 0.0034 0.0165 tensor([0.4567])
9 0.0032 0.0189 tensor([0.4302])
10 0.0031 0.0190 tensor([0.4309])
11 0.0030 0.0163 tensor([0.4740])
12 0.0028 0.0192 tensor([0.4481])
13 0.0026 0.0158 tensor([0.4978])
14 0.0025 0.0181 tensor([0.4761])
15 0.0024 0.0172 tensor([0.4919])
16 0.0023 0.0165 tensor([0.5044])
17 0.0022 0.0172 tensor([0.4988])
18 0.0021 0.0185 tensor([0.4883])
19 0.0021 0.0178 tensor([0.5000])
20 0.0020 0.0183 tensor([0.5007])
21 0.0019 0.0189 tensor([0.4982])
22 0.0019 0.0161 tensor([0.5300])
23 0.0018 0.0181 tensor([0.5162])
24 0.0018 0.0163 tensor([0.5328])
25 0.0017 0.0158 tensor([0.5378])
26 0.0017 0.0157 tensor([0.5506])
27 0.0016 0.0171 tensor([0.5302])
28 0.0016 0.0163 tensor([0.5455])
29 0.0015 0.0159 tensor([0.5463]